# Chapter 19: Training and Deploying TensorFlow Models at Scale

Pada chapter ini, fokus pembahasan Machine Learning bergeser dari sekadar membangun model yang akurat
menjadi bagaimana model tersebut dapat digunakan secara nyata di lingkungan produksi berskala besar.

Dalam praktik industri, sebuah model Machine Learning tidak hanya dinilai dari akurasinya,
tetapi juga dari kemampuannya untuk:
- berjalan stabil dalam jangka panjang,
- melayani banyak permintaan secara bersamaan,
- mudah diperbarui ketika data berubah,
- dan terintegrasi dengan sistem lain.

Oleh karena itu, chapter ini membahas aspek penting seperti deployment, model serving,
versioning, scalability, serta penggunaan hardware dan cloud infrastructure.


## 1. Dari Model Eksperimental ke Model Produksi

Model Machine Learning yang dilatih di notebook atau script Python umumnya masih berada
pada tahap eksperimen. Model seperti ini biasanya hanya diuji menggunakan dataset statis
dan dijalankan di satu mesin.

Sebaliknya, model produksi harus siap menghadapi kondisi dunia nyata, seperti:
- data yang terus berubah,
- permintaan pengguna secara real-time,
- keterbatasan latensi,
- serta potensi kegagalan sistem.

Transisi dari model eksperimental ke model produksi membutuhkan pendekatan yang lebih sistematis,
tidak hanya dari sisi model, tetapi juga dari sisi software engineering dan infrastruktur.


## 2. Model Serving sebagai Layanan

Dalam sistem Machine Learning modern, model jarang dijalankan langsung di dalam aplikasi utama.
Sebagai gantinya, model biasanya dijalankan sebagai layanan terpisah (model serving service).

Pendekatan ini memungkinkan:
- aplikasi mengakses model melalui API,
- penggantian model tanpa mengubah aplikasi,
- dan skalabilitas sistem yang lebih baik.

Model serving sering diimplementasikan menggunakan REST API atau gRPC,
sehingga dapat diakses oleh berbagai bahasa pemrograman dan platform.


## 3. Tantangan Production Machine Learning

Deployment model Machine Learning ke production membawa tantangan baru yang tidak ditemui
saat eksperimen, antara lain:

- Model versioning untuk mengelola banyak versi model
- Rollback jika model baru mengalami kegagalan
- A/B testing untuk membandingkan performa model
- Scalability untuk menangani lonjakan trafik
- Monitoring untuk mendeteksi degradasi performa

Tanpa perencanaan yang baik, sistem Machine Learning dapat menjadi sulit dipelihara
dan berisiko mengganggu layanan bisnis.


## 4. Gambaran Umum Chapter 19

Chapter ini membahas berbagai strategi deployment dan scaling model TensorFlow, termasuk:
- penyimpanan model menggunakan format standar,
- deployment menggunakan TensorFlow Serving,
- deployment ke cloud dan edge devices,
- optimisasi performa menggunakan hardware acceleration,
- serta training terdistribusi.

Keseluruhan topik ini merupakan fondasi penting bagi Machine Learning skala industri.


## 5. Saving and Exporting Models

Sebelum sebuah model dapat digunakan di production, model tersebut harus disimpan
dalam format yang stabil, portabel, dan dapat digunakan lintas platform.

TensorFlow menyediakan format resmi bernama SavedModel yang dirancang khusus
untuk kebutuhan deployment dan serving.


### 5.1 Mengapa Tidak Menyimpan Model sebagai File Python?

Menyimpan model hanya sebagai file Python memiliki berbagai keterbatasan, seperti:
- ketergantungan pada environment Python tertentu,
- sulit digunakan oleh sistem non-Python,
- tidak menyimpan graph komputasi secara eksplisit.

Lingkungan produksi membutuhkan format model yang bersifat language-agnostic
dan dapat dijalankan tanpa kode training.


### 5.2 SavedModel Format

SavedModel adalah format standar TensorFlow yang menyimpan:
- arsitektur model,
- bobot (weights),
- computation graph,
- dan signature untuk inference.

Format ini digunakan oleh TensorFlow Serving, TensorFlow Lite, dan TensorFlow.js,
sehingga menjadi pusat ekosistem deployment TensorFlow.


### 5.3 Menyimpan Model dengan Keras

Keras menyediakan method save() yang secara otomatis menyimpan model
dalam format SavedModel, sehingga siap digunakan untuk deployment.


In [ ]:
model.save("my_saved_model")


### 5.4 Memuat Kembali Model

Model yang telah disimpan dapat dimuat kembali tanpa perlu mendefinisikan ulang
arsitektur atau bobot, sehingga memudahkan pemisahan antara training dan inference.


In [ ]:
import tensorflow as tf

loaded_model = tf.keras.models.load_model("my_saved_model")


### 5.5 Model Signatures

Model signature mendefinisikan bagaimana model menerima input dan menghasilkan output.
Signature ini sangat penting untuk integrasi model dengan REST API, gRPC,
dan sistem eksternal lainnya.


## 6. TensorFlow Serving

TensorFlow Serving adalah sistem khusus yang dirancang untuk menyajikan
model TensorFlow di lingkungan produksi dengan latensi rendah dan throughput tinggi.


### 6.1 Arsitektur TensorFlow Serving

TensorFlow Serving berjalan sebagai server mandiri yang memuat model dari disk,
mengelola versi model, dan menangani request inference dari klien.


### 6.2 Menjalankan TensorFlow Serving dengan Docker

Docker digunakan untuk memastikan environment deployment konsisten
dan mudah direplikasi di berbagai sistem.
Cara paling praktis untuk menjalankan TensorFlow Serving adalah menggunakan Docker image resmi.

Docker memastikan:

    environment konsisten,
    dependensi terkelola,
    deployment yang mudah direplikasi.


In [ ]:
docker run -p 8501:8501 \
  --mount type=bind,source=/path/to/my_saved_model,target=/models/my_model \
  -e MODEL_NAME=my_model \
  tensorflow/serving


### 6.3 REST API untuk Inference

REST API memungkinkan klien melakukan inference menggunakan HTTP dan JSON,
sehingga mudah diintegrasikan dengan berbagai aplikasi.


In [ ]:
import requests
import json

data = json.dumps({
    "instances": X_test[:1].tolist()
})

response = requests.post(
    "http://localhost:8501/v1/models/my_model:predict",
    data=data
)

print(response.json())


### 6.4 gRPC API

gRPC menawarkan performa yang lebih tinggi dibanding REST,
terutama untuk sistem dengan trafik besar dan latensi rendah.
Namun, implementasinya lebih kompleks.


## 7. Model Versioning

Model Machine Learning perlu diperbarui secara berkala seiring perubahan data.
Oleh karena itu, sistem produksi harus mendukung pengelolaan banyak versi model.


### 7.1 Versioning pada TensorFlow Serving

TensorFlow Serving menggunakan struktur direktori bernomor untuk versioning model,
sehingga versi lama tetap tersedia dan rollback dapat dilakukan dengan cepat.


## 8. A/B Testing untuk Model

A/B testing memungkinkan dua model dijalankan secara paralel di production
untuk membandingkan performa berdasarkan data pengguna nyata.


## 9. Rollback Strategy

Dalam lingkungan produksi, deployment model Machine Learning tidak selalu berjalan mulus.
Model baru yang secara offline terlihat lebih baik bisa saja menimbulkan masalah ketika
digunakan pada data nyata.

Beberapa contoh masalah yang sering terjadi setelah deployment model baru:
- penurunan performa prediksi akibat data distribution shift,
- peningkatan latensi inference yang mengganggu user experience,
- bug atau kesalahan preprocessing yang tidak terdeteksi saat training,
- dampak negatif terhadap metrik bisnis meskipun akurasi meningkat.

Karena itu, sistem Machine Learning produksi **harus memiliki mekanisme rollback**
yang cepat, aman, dan minim gangguan layanan.


Rollback strategy bertujuan untuk:
- mengembalikan sistem ke kondisi stabil sebelumnya,
- meminimalkan downtime,
- melindungi sistem bisnis dari dampak model yang bermasalah.

Tanpa rollback yang baik, kesalahan model dapat menyebabkan kerugian finansial,
hilangnya kepercayaan pengguna, atau gangguan operasional.


Dengan TensorFlow Serving, rollback dapat dilakukan dengan sangat efisien karena:
- setiap versi model disimpan secara terpisah,
- server dapat memilih versi aktif tanpa restart penuh,
- trafik dapat langsung dialihkan ke model lama.

Pendekatan ini membuat rollback menjadi proses operasional,
bukan proses pengembangan ulang model.


## 10. Cloud Deployment

Pada skala besar, menjalankan model Machine Learning di server lokal sering kali
tidak cukup untuk memenuhi kebutuhan performa, ketersediaan, dan skalabilitas.

Cloud deployment memungkinkan sistem Machine Learning:
- menangani jutaan request per hari,
- tetap tersedia meskipun terjadi kegagalan hardware,
- menyesuaikan resource secara otomatis sesuai beban kerja.

Karena alasan inilah, mayoritas sistem Machine Learning di industri modern
mengandalkan cloud infrastructure.


Cloud deployment juga memungkinkan integrasi yang lebih erat dengan:
- pipeline data,
- sistem monitoring dan logging,
- layanan keamanan dan autentikasi,
- serta workflow MLOps.

Dengan demikian, cloud bukan hanya tempat menjalankan model,
tetapi menjadi bagian dari ekosistem Machine Learning secara keseluruhan.


### 10.1 Mengapa Menggunakan Cloud untuk ML?

Penggunaan cloud dalam Machine Learning menawarkan berbagai keuntungan strategis,
terutama untuk sistem yang terus berkembang.

Beberapa alasan utama penggunaan cloud untuk ML antara lain:
- autoscaling untuk menghadapi lonjakan trafik,
- ketersediaan tinggi (high availability),
- kemudahan deployment lintas wilayah,
- pengurangan biaya infrastruktur jangka panjang,
- fokus tim pada pengembangan model, bukan server.


Selain itu, cloud juga mendukung kolaborasi tim yang lebih baik,
karena model, data, dan pipeline dapat diakses secara terpusat.

Pendekatan ini sangat cocok untuk organisasi yang ingin
membangun sistem ML yang berkelanjutan dan siap tumbuh.


### 10.2 AI Platform / Vertex AI Overview

Google Cloud AI Platform (sekarang dikenal sebagai Vertex AI)
adalah layanan managed Machine Learning yang menyediakan
solusi end-to-end untuk seluruh siklus hidup model.

Vertex AI mendukung:
- training model skala besar,
- hyperparameter tuning otomatis,
- deployment model ke endpoint,
- monitoring performa model secara real-time.


Dengan Vertex AI, pengembang tidak perlu membangun
sistem serving, autoscaling, dan monitoring dari nol.

Hal ini mempercepat proses deployment sekaligus
mengurangi kompleksitas operasional dalam jangka panjang.


### 10.3 Menyiapkan Model untuk Cloud Deployment

Sebelum model dapat dideploy ke cloud,
model harus disimpan dalam format yang kompatibel
dengan layanan managed serving.

Format SavedModel digunakan karena:
- menyimpan arsitektur dan bobot model,
- bersifat portable lintas platform,
- didukung penuh oleh layanan cloud TensorFlow.


Model biasanya diunggah ke Cloud Storage
dan disusun dalam struktur direktori versi,
sehingga memudahkan manajemen dan rollback.


### 10.4 Deploying Model to AI Platform

Setelah model berada di Cloud Storage,
model dapat dideploy sebagai endpoint inference.

Endpoint ini:
- memuat model secara otomatis,
- menangani request dari berbagai klien,
- melakukan autoscaling sesuai beban kerja.


Deployment dapat dilakukan melalui:
- Google Cloud Console,
- command-line interface (gcloud),
- atau pipeline CI/CD berbasis MLOps.

Pendekatan ini memisahkan deployment dari kode aplikasi,
sehingga sistem lebih fleksibel dan mudah dikelola.


### 10.5 Keuntungan Managed ML Services

Menggunakan layanan managed ML memberikan berbagai keuntungan,
terutama bagi tim dengan sumber daya terbatas.

Keuntungan utama meliputi:
- pengelolaan infrastruktur otomatis,
- autoscaling dan load balancing bawaan,
- integrasi logging dan monitoring,
- keamanan tingkat enterprise.


Namun, pendekatan ini juga memiliki beberapa trade-off,
seperti biaya layanan cloud dan ketergantungan pada vendor tertentu.

Karena itu, pemilihan managed service harus disesuaikan
dengan kebutuhan bisnis dan skala sistem.


## 11. Deployment ke Mobile & Embedded Devices

Tidak semua aplikasi Machine Learning dijalankan di server cloud.
Dalam banyak kasus, model perlu dijalankan langsung di perangkat pengguna,
seperti smartphone atau perangkat IoT.

Pendekatan ini dikenal sebagai on-device atau edge Machine Learning.


Deployment di perangkat lokal memungkinkan:
- latensi yang sangat rendah,
- perlindungan privasi data,
- pengurangan ketergantungan pada koneksi internet,
- efisiensi biaya server.


### 11.1 Mengapa On-Device ML?

On-device ML sangat penting untuk aplikasi real-time
dan aplikasi yang membutuhkan respons instan.

Beberapa contoh penggunaan:
- face recognition di smartphone,
- voice assistant offline,
- analisis sensor IoT,
- kamera pintar.


Namun, perangkat memiliki keterbatasan komputasi,
sehingga model harus dioptimalkan secara khusus.


### 11.2 TensorFlow Lite

TensorFlow Lite adalah framework TensorFlow versi ringan
yang dirancang khusus untuk perangkat dengan resource terbatas.

TFLite menyediakan runtime inference yang cepat,
ukuran model kecil, dan dukungan hardware acceleration.


### 11.3 Mengonversi Model ke TensorFlow Lite

Model TensorFlow standar dapat dikonversi ke format TFLite
menggunakan TFLiteConverter.


In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model("my_saved_model")
tflite_model = converter.convert()

with open("model.tflite", "wb") as f:
    f.write(tflite_model)


### 11.4 Model Optimization

Agar model dapat berjalan optimal di perangkat terbatas,
diperlukan teknik optimisasi seperti:
- quantization,
- pruning,
- operator fusion.


Optimisasi ini dapat mengurangi ukuran model
dan mempercepat inference dengan penurunan akurasi minimal.


### 11.5 Use Cases TensorFlow Lite

TensorFlow Lite banyak digunakan pada:
- image classification mobile,
- speech recognition offline,
- computer vision embedded system,
- sensor data processing.


## 12. Deployment di Browser dengan TensorFlow.js

TensorFlow.js memungkinkan model Machine Learning
dijalankan langsung di browser tanpa server inference.

Pendekatan ini membuka peluang aplikasi ML berbasis web
yang interaktif dan real-time.


### 12.1 Keuntungan Deployment di Browser

Deployment di browser menawarkan latensi rendah,
privasi data, dan kemudahan distribusi aplikasi ML.


### 12.2 TensorFlow.js Overview

TensorFlow.js adalah library JavaScript untuk menjalankan
dan melatih model Machine Learning di browser.


### 12.3 Mengonversi Model ke TensorFlow.js

Model TensorFlow dapat dikonversi ke format TensorFlow.js
menggunakan tensorflowjs_converter.


In [ ]:
tensorflowjs_converter \
  --input_format=tf_saved_model \
  --output_format=tfjs_graph_model \
  my_saved_model \
  tfjs_model


### 12.4 Menggunakan Model di JavaScript

Model TensorFlow.js dapat dimuat langsung di JavaScript
untuk inference berbasis web.


### 12.4 Menggunakan Model di JavaScript

Model TensorFlow.js dapat dimuat langsung di JavaScript
untuk inference berbasis web.


In [ ]:
const model = await tf.loadGraphModel('tfjs_model/model.json');
const prediction = model.predict(inputTensor);


## 13. Hardware Acceleration

Seiring bertambahnya ukuran model dan dataset,
CPU saja sering kali tidak cukup untuk training dan inference.

TensorFlow mendukung akselerasi menggunakan GPU dan TPU
untuk meningkatkan performa secara signifikan.


### 13.1 GPU Acceleration

GPU sangat efektif untuk neural network
karena mampu menjalankan ribuan operasi paralel.


### 13.2 TPU (Tensor Processing Unit)

TPU adalah hardware khusus dari Google
yang dioptimalkan untuk workload TensorFlow skala besar.


## 14. Distributed Training

Distributed training memungkinkan training model
dilakukan secara paralel pada banyak GPU atau mesin.


TensorFlow menyediakan tf.distribute.Strategy untuk mempermudah distributed training.

Beberapa strategi yang umum digunakan:

    MirroredStrategy: training paralel pada banyak GPU di satu mesin,
    MultiWorkerMirroredStrategy: training pada banyak mesin,
    TPUStrategy: training pada TPU.

TensorFlow menyediakan tf.distribute.Strategy
untuk mempermudah implementasi distributed training.


In [ ]:
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=[28, 28]),
        tf.keras.layers.Dense(300, activation="relu"),
        tf.keras.layers.Dense(100, activation="relu"),
        tf.keras.layers.Dense(10, activation="softmax")
    ])

    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer="sgd",
        metrics=["accuracy"]
    )


## Closing Summary (Chapter 19)

Chapter ini membahas aspek paling krusial dari Machine Learning modern,
yaitu bagaimana membawa model dari tahap eksperimen ke sistem produksi berskala besar.

Keberhasilan Machine Learning di dunia nyata tidak hanya ditentukan oleh
akurasi model, tetapi juga oleh stabilitas, skalabilitas, dan kemudahan pemeliharaan.

Dengan memahami deployment, serving, cloud infrastructure, hardware acceleration,
dan distributed training, praktisi Machine Learning dapat membangun sistem
yang benar-benar siap digunakan di lingkungan industri.
